In [14]:
import gradio as gr
import time
import ollama

In [23]:
# 1. 고정된 장면 설정
scenes = [
    ("헬리콥터가 격추된 뒤 해변에 추락했습니다. 포탄이 빗발치는 속에서 첫 전투를 준비해야 합니다.",
     "기계 생명체들은 엄폐물이 없는 해변을 초토화하고 있습니다. 빠르게 은폐처를 찾아야 하며, 무분별한 돌진은 즉사로 이어질 수 있습니다."),
    ("기계 생명체들이 매복해 있던 참호에 도착했습니다. 팀원들은 혼란에 빠져 사방으로 흩어졌습니다.",
     "참호를 따라 빠르게 이동하거나, 근접한 기계 생명체를 정밀 사격해 제거해야 합니다. 무턱대고 머물면 포위될 위험이 큽니다."),
    ("상관이 전사하고, 지휘 체계는 완전히 붕괴되었습니다. 이제 당신 혼자입니다.",
     "통신 장비를 복구해 지원을 요청하거나, 적 경로를 피해 은밀하게 이동해야 합니다. 무작정 교전하면 압도적인 수에 휩쓸립니다."),
    ("주요 공격 목표였던 적 통신탑이 완전히 점령당했습니다. 지원군도 오지 않고 있습니다.",
     "통신탑의 방어 드론을 우회하거나 교란 장치를 설치해야 합니다. 정면 돌파는 실패 확률이 매우 높습니다."),
    ("최종 본진에 도착했습니다. 알파 개체, 적 지휘 핵심이 모습을 드러냈습니다.",
     "알파 개체는 직접 공격으로는 쓰러지지 않습니다. 주의를 끌어 시간을 벌거나 약점을 찾아야 합니다. 무력 돌파는 자살행위입니다.")
]

In [24]:
# 게임 상태 초기화
game_state = {
    "scene": 0,
    "memory": [],
    "loop": 0,
    "game_over": False
}

def reset_game_state():
    game_state["scene"] = 0
    game_state["memory"].clear()
    game_state["loop"] = 0
    game_state["game_over"] = False

In [25]:
# 현재 장면 출력 + 생존/사망 판정 함
def get_current_scene():
    if game_state["scene"] < len(scenes):
        main_text, sub_text = scenes[game_state["scene"]]
        return f"{main_text}\n{sub_text}"
    else:
        boss_main = "거대한 오메가 기계가 모습을 드러냈습니다. 붉게 빛나는 촉수들이 전장을 뒤덮고, 모든 공격이 당신을 향하고 있습니다."
        boss_sub = "시간이 없습니다. 몇 초 안에 움직이지 않으면 전장이 그대로 얼어붙을 것입니다. 마지막 결단을 내려야 합니다."
        return f"{boss_main}\n{boss_sub}"

def ask_ollama_for_survival(scene_text, user_action):
    system_prompt = """
당신은 Edge of Tomorrow 전장의 상황을 기반으로 플레이어의 행동이 생존으로 이어질지 사망으로 이어질지 판정하는 AI입니다.
응답은 반드시 "생존" 또는 "사망" 한 단어로만 답하세요.
"""
    user_prompt = f"상황: {scene_text}\n플레이어의 행동: {user_action}\n이 행동은 생존입니까, 사망입니까?"

    response = ollama.chat(
        model="EEVE-Korean-10.8B",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]
    )
    return response['message']['content'].strip()


In [26]:
# 행동 처리 함
def process_input(user_input):
    if game_state["game_over"]:
        return "🎉 게임이 종료되었습니다. 다시 시작하려면 초기화 버튼을 누르세요."

    current_scene_text = get_current_scene()

    if game_state["scene"] >= len(scenes):
        boss_scene_text = get_current_scene()
        survival_result = ask_ollama_for_survival(boss_scene_text, user_input)
        if survival_result == "생존":
            game_state["game_over"] = True
            return "👑 당신은 오메가를 쓰러뜨리고 인류를 구했습니다! 전장을 정복했습니다. 🎉"
        else:
            game_state["loop"] += 1
            game_state["scene"] = 0
            game_state["memory"].append(("보스전", "죽음"))
            return "☠️ 오메가에게 패배했습니다. 루프가 다시 시작됩니다."

    survival_result = ask_ollama_for_survival(current_scene_text, user_input)

    if survival_result == "생존":
        game_state["scene"] += 1
        return "✅ 안전하게 다음 장면으로 넘어갑니다!"
    else:
        game_state["loop"] += 1
        game_state["scene"] = 0
        game_state["memory"].append((user_input, "죽음"))
        return "☠️ 행동 실패로 사망했습니다. 루프가 다시 시작됩니다."


In [27]:
with gr.Blocks() as demo:
    # ⬇️ Fade-out CSS를 삽입하는 숨은 HTML 컴포넌트
    gr.HTML('''
    <style>
    @keyframes fadeOut {
        from {opacity: 1;}
        to {opacity: 0;}
    }
    #result {
        animation: fadeOut 1s ease-out 1s forwards;
    }
    </style>
    ''')

    with gr.Row():
        output_box = gr.Textbox(label="🧠 전장의 상황", lines=6, interactive=False)
    with gr.Row():
        result_box = gr.HTML(label="🎯 결과")
    with gr.Row():
        user_input = gr.Textbox(label="당신의 행동을 입력하세요")
    with gr.Row():
        submit_btn = gr.Button("결정!")
        reset_btn = gr.Button("🔄 게임 다시 시작하기")

    def on_submit(input_text):
        scene_description = get_current_scene()
        result = process_input(input_text)
        
        html_result = f'''
        <div id="result" style="font-size:20px;">
            {result}
        </div>
        '''
        return scene_description, html_result

    def on_reset():
        reset_game_state()
        return get_current_scene(), ""

    submit_btn.click(on_submit, inputs=[user_input], outputs=[output_box, result_box])
    reset_btn.click(on_reset, outputs=[output_box, result_box])

    demo.load(on_reset, outputs=[output_box, result_box])

In [28]:
demo.launch(share=True)

* Running on local URL:  http://127.0.0.1:7868
* Running on public URL: https://3c6c0620ff85384720.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
